# Paradex vs. Lighter Price Spread Analysis (Mainnet Realtime)

This notebook fetches minute-level price data from **Paradex Futures (Mainnet)** and **Lighter (Mainnet)** to analyze the spread.

**Objective:** Plot `(Paradex Price - Lighter Price) / Paradex Price` in real-time.
> **Note:** We use public endpoints for data fetching, so no API keys are required.

In [ ]:
import os
import sys
import asyncio
import time
import aiohttp
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from IPython.display import clear_output, display

# Add src to path
sys.path.append(os.path.abspath('src'))

%matplotlib inline

In [ ]:
# 1. Initialize Constants

# Lighter Mainnet API URL
LIGHTER_MAINNET_API = "https://mainnet.zklighter.elliot.ai"
LIGHTER_API_VERSION = "/api/v1" 

# Paradex Mainnet API URL
PARADEX_MAINNET_API = "https://api.prod.paradex.trade/v1"

# Helper to fetch public data without SDK overhead
async def fetch_public_json(url, params=None):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "application/json"
    }
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.get(url, params=params) as resp:
            if resp.status == 200:
                return await resp.json()
            else:
                # text = await resp.text()
                # print(f"Error {resp.status} fetching {url}: {text}")
                return None

print("Clients Initialized (Mainnet).")

## 2. Real-Time Monitor
Run the cell below to start the live plotting. Stop it by clicking 'Stop' or Interrupt Kernel.

In [ ]:
async def live_spread_monitor():
    data = []
    print(f"Starting Paradex vs Lighter Real-Time Monitor...")
    
    # Setup Plot: 3 rows now
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 12), sharex=True)
    
    try:
        while True:
            loop_start = time.time()
            now = datetime.now()
            
            # 1. Fetch Data
            p_price = None
            l_mid = None
            
            # Paradex (v1 markets summary)
            try:
                # /v1/markets/summary?market=ETH-USD-PERP is public
                url = f"{PARADEX_MAINNET_API}/markets/summary"
                params = {'market': 'ETH-USD-PERP'}
                resp = await fetch_public_json(url, params)
                
                # Response format: { "results": [ { "symbol": "ETH-USD-PERP", "last_traded_price": "...", ... } ] }
                if resp and 'results' in resp and len(resp['results']) > 0:
                     market_data = resp['results'][0]
                     p_price = float(market_data['last_traded_price'])
            except Exception as e:
                # print(f"Paradex Error: {e}")
                pass
            
            # Lighter Price (Fetch Orderbook)
            try:
                ob_url = f"{LIGHTER_MAINNET_API}{LIGHTER_API_VERSION}/orderBookOrders"
                # market_id=2048 for ETH/USDC
                market_id = 2048 
                params = {'market_id': market_id, 'limit': 1}
                l_ob = await fetch_public_json(ob_url, params)

                if l_ob and 'bids' in l_ob and 'asks' in l_ob:
                    if len(l_ob['bids']) > 0 and len(l_ob['asks']) > 0:
                         bid_entry = l_ob['bids'][0]
                         ask_entry = l_ob['asks'][0]
                         bid_price = float(bid_entry['price']) if isinstance(bid_entry, dict) else float(bid_entry[0])
                         ask_price = float(ask_entry['price']) if isinstance(ask_entry, dict) else float(ask_entry[0])
                         l_mid = (bid_price + ask_price) / 2
            except: pass
            
            # 2. Store & Calculate
            if p_price and l_mid:
                spread = (p_price - l_mid) / p_price
                data.append({'timestamp': now, 'paradex': p_price, 'lighter': l_mid, 'spread': spread})
                
                # Keep last 1000 points
                if len(data) > 1000: data.pop(0)

            # 3. Update Plot (every ~2nd tick to avoid flicker)
            if len(data) > 4:
                df = pd.DataFrame(data).set_index('timestamp')
                
                # Calculate 10s Moving Average
                df['spread_10s'] = df['spread'].rolling('10s').mean()
                
                clear_output(wait=True)
                
                # Reset Axes
                ax1.cla()
                ax2.cla()
                ax3.cla()
                
                # Price Chart
                ax1.plot(df.index, df['paradex'], label='Paradex', color='purple')
                ax1.plot(df.index, df['lighter'], label='Lighter', color='blue')
                ax1.legend(loc='upper left')
                ax1.set_title(f"[Live] ETH Price | Paradex: {p_price:.2f} | Lighter: {l_mid:.2f}")
                ax1.grid(True, alpha=0.3)
                
                # Spread Chart
                ax2.plot(df.index, df['spread'], label='Spread', color='green', alpha=0.6)
                ax2.axhline(0, color='black', linestyle='--', alpha=0.5)
                ax2.legend(loc='upper left')
                ax2.set_title(f"Rel. Spread: {spread*10000:.2f} bps")
                ax2.grid(True, alpha=0.3)
                
                # Avg Spread Chart (New)
                latest_avg = df['spread_10s'].iloc[-1]
                ax3.plot(df.index, df['spread_10s'], label='10s Avg Spread', color='red', linewidth=2)
                ax3.axhline(0, color='black', linestyle='--', alpha=0.5)
                ax3.legend(loc='upper left')
                ax3.set_title(f"10s Avg Spread: {latest_avg*10000:.2f} bps")
                ax3.grid(True, alpha=0.3)
                
                display(fig)
            
            # Sleep remainder of 1s interval
            elapsed = time.time() - loop_start
            if elapsed < 1.0:
                await asyncio.sleep(1.0 - elapsed)
                
    except KeyboardInterrupt:
        print("Stopped by User.")
        plt.close()


await live_spread_monitor()